In [ ]:
#%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

In [ ]:
# from fastai.column_data import *
# from fastai.conv_learner import *
# from fastai.dataset import *
from fastai.torch_imports import save_model, load_model

In [ ]:
from models import *
from utils import *

In [ ]:
ls -f KITTI/ | grep .csv

In [ ]:
train, val = pd.read_csv('./KITTI/training_192_640_pre.csv'), pd.read_csv('./KITTI/validation_192_640_pre.csv')
train, val = shuffle(train), shuffle(val);

In [ ]:
train.head()

In [ ]:
val.head()

In [ ]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [ ]:
trn_camera.head()

In [ ]:
arch = f
bs = 4
PATH = 'Fastai_TRN'
DPATH = "KITTI/"
verbose = 500

In [ ]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [ ]:
stats = imagenet_stats
tfm_norm = Normalize(*stats, tfm_y=TfmType.NO) 
tfm_denorm = Denormalize(*stats)

In [ ]:
trn_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)
val_tfms = UnScaleTransforms(sz=None, tfms=[], normalizer=tfm_norm, denorm=tfm_denorm, tfm_y=TfmType.NO, sz_y=None)

In [ ]:
tfms = (trn_tfms, val_tfms)

MD1 = get_MD(trn0, val0, tfms, bs, DPATH, PATH)
MD2 = get_MD(trn1, val1, tfms, bs, DPATH, PATH)
MD3 = get_MD(trn2, val2, tfms, bs, DPATH, PATH)
MDcam = get_cam(trn_camera, val_camera, bs)

In [ ]:
imgnet_mean, imgnet_std = torch.from_numpy(stats[0]).float(), torch.from_numpy(stats[1]).float()
imgnet_mean, imgnet_std = imgnet_mean.view(1,3,1,1), imgnet_std.view(1,3,1,1)
denorm = denormer(imgnet_mean, imgnet_std)

In [ ]:
class Loss(nn.Module):
    def __init__(self, scale=1, Tscale=0.15):
        super().__init__()
        self.appr = TriAppearanceLoss(scale=Tscale) 
        self.smooth = SmoothLoss()
        self.scale = scale
    def forward(self, d1s, d2s ,d3s, trans, rotation, x1, x2, x3, camera):
        appr_loss, details = self.appr(d2s, trans, rotation, x1, x2, x3, camera)
        smooth_losses = [ 0.5/(i+1) * self.smooth(d2) for i, d2 in enumerate(d2s) ]
        #smooth_losses = [ self.smooth(d1)+self.smooth(d2) + self.smooth(d3) for d1, d2, d3 in zip(d1s, d2s, d3s) ]
        smooth_loss = torch.mean(torch.cat(smooth_losses, dim=0)) * self.scale
        #print(type(appr_loss))
        #print(type(smooth_loss))
        return appr_loss + smooth_loss, (appr_loss, smooth_loss, *details) 
               

In [ ]:
def train(m, opt, MD1, MD2, MD3, MDcam, denorm):
    m.train()
    DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
    losses = []  
    
    #for i in range(2): # just for testing
    for i in range(len(MD1.trn_ds)//bs-len(losses)):
        opt.zero_grad()
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])

        d1, d2, d3, trans, rotation, = m(imgs1, imgs2, imgs3)
        #pdb.set_trace()
        imgs1, imgs2, imgs3 = denorm(imgs1), denorm(imgs2), denorm(imgs3) 
        #pdb.set_trace()
        loss, details = l(d1, d2, d3, trans, rotation, imgs1, imgs2, imgs3, cam)
        loss.backward()
        opt.step()
        losses.append(loss.data[0])
        # appr smooth ssim l1
        if i%verbose == 0: print(loss.data[0],
                                 details[0].data[0],
                                 details[1].data[0],
                                 details[2].data[0],
                                 details[3].data[0],
                                 sep='\t')
    return losses

In [ ]:
def evaluate(m, MD1, MD2, MD3, MDcam, folder):
    
    DL1, DL2, DL3, DLcam = iter(MD1.val_dl), iter(MD2.val_dl), iter(MD3.val_dl), iter(MDcam.val_dl)
    m.eval()
    for i in range(10): 
        imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])
        d1s, d2s, d3s, trans, rotation, = m(imgs1, imgs2, imgs3)
        imgs1, imgs2, imgs3 = denorm(imgs1), denorm(imgs2), denorm(imgs3) 
       
        i = 0
        d2 = d2s[i]
        if i>0: d2 = F.upsample(input=d2, scale_factor=2**i, mode='bilinear')
        cx12, cy12, dm12 = l.appr.offset.forward(trans=trans[:,0], rotation=rotation[:,0], inv_depth = d2, camera = cam)
        x12, ivm12 = l.appr.sampler.forward(imgs1, cx12, cy12)
        
        imgs2 = tonp(imgs2.permute(0,2,3,1))
        x12 = tonp(x12.permute(0,2,3,1))
        d2 = tonp(d2)
        
        for j in range(bs):
            save_res(imgs2[i], x12[i], d2[i][0], folder/"{}{}.png".format(i,j))

In [ ]:
def fit(epoch, m, opt, MD1, MD2, MD3, MDcam):
    for i in range(epoch):
        print("--------------------epoch {} start:----------------------".format(i))
        losses = train(m, opt, MD1, MD2, MD3, MDcam)
        losses= [ str(loss) for loss in losses ]
        folder = Path("./tmp")
        folder.mkdir(exist_ok=True)
        file = folder / 'epoch{}.log'.format(i)
        with file.open('w') as f:
            f.write("\n".join(losses))
        save_model(m, str(folder / "epoch{}.M".format(i)))

In [ ]:
TS = [0, 1, 0.15, 0.3, 0.5, 0.75]

In [ ]:
expfolder = Path('./experiment/loss/')
expfolder.mkdir(parents=True, exist_ok=True)

In [ ]:
for ts in TS:
    thisfolder = expfolder / "tscale_{}".format(ts)
    thisfolder.mkdir(exist_ok=True)
    
    m = TriDepth(get_base(), 1).cuda()
    set_trainable(m.depth.rn, False)
    l = Loss(scale=1, Tscale=ts).cuda()
    opt = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr =1e-4)
    print("training model")
    losses = train(m, opt, MD1, MD2, MD3, MDcam, denorm)
    print("saving model")
    save_model(m, thisfolder/"model.M".format(ts))
    print("writting log")
    logger = thisfolder/"log"
    losses = [ str(loss) for loss in losses]
    with logger.open('w') as f: f.write("\n".join(losses))
    print("predicting")    
    predictfolder = thisfolder/'predicts'
    predictfolder.mkdir(exist_ok=True)
    evaluate(m, MD1, MD2, MD3, MDcam, predictfolder)

In [ ]:
m = TriDepth(get_base(), 1).cuda()
set_trainable(m.depth.rn, False)
l = Loss(scale=1, Tscale=0.15).cuda()
opt = optim.Adam(filter(lambda p: p.requires_grad, m.parameters()), lr =1e-4)


In [ ]:
m.train()

In [ ]:
DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)

In [ ]:
losses = []  
    

In [ ]:
opt.zero_grad()

In [ ]:
imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])

In [ ]:
d1, d2, d3, trans, rotation, = m(imgs1, imgs2, imgs3)

In [ ]:
imgs1, imgs2, imgs3 = denorm(imgs1), denorm(imgs2), denorm(imgs3) 

In [ ]:
loss, details = l(d1, d2, d3, trans, rotation, imgs1, imgs2, imgs3, cam)

In [ ]:
loss.backward()

In [ ]:
opt.step()

In [ ]:
losses.append(loss.data[0])

In [ ]:
# appr smooth ssim l1
if i%verbose == 0: print(loss.data[0],
                         details[0].data[0],
                         details[1].data[0],
                         details[2].data[0],
                         details[3].data[0],
                         sep='\t')